In [35]:
import psycopg2
import websim

In [72]:
conn = psycopg2.connect(
    dbname = 'AlphaDB', 
    user = 'JohnnyReb', 
    port = 5432, 
    host = 'alphadb1.ccdpeu6icdye.us-west-2.rds.amazonaws.com', 
    password = 'Mk9HEEr0?7')

In [73]:
cur = conn.cursor()

In [1]:
def add_code(code, comment = '', author = 'python'):
    cur.execute('INSERT INTO code (code, comment, author) VALUES (%s, %s, %s)', (code, comment, author))
def get_codes():
    cur.execute('SELECT * FROM code')
    for row in cur.fetchall():
        print row
def add_sending(codeID, params = None):
    cur.execute('INSERT INTO sending (codeID, params) VALUES (%s, %s)', (codeID, params))
    
def get_sending_to_exec():
    cur.execute('''
        SELECT 
            code.code,
            code.params_info,
            sending.params,
            sending.ID
        FROM
            sending JOIN code ON sending.codeID = code.ID
        where
            sending.ID not in (SELECT sendingID FROM processing)
        limit 1
    ''')
    sending = cur.fetchone()
    return sending

def get_websim(account_id):
    cur.execute('''
        SELECT 
            login,
            password,
            name,
            comment
        FROM
            websim_account
        where
            ID = %s 
        limit 1
    ''', (account_id, ))
    account = cur.fetchone()
    print account
    ws = websim.WebSim(login = account[0], password = account[1])
    return ws

def process_sending(sending, account_id):
    def set_status(status):
        cur.execute('INSERT INTO processing (sendingID, status, websim_accID) VALUES (%s, %s, %s)', (sending[3], status, account_id))
        conn.commit()
    set_status('processing')
    ws = get_websim(account_id)
    ws.authorise()
    res = ws.send_alpha(sending[0]) # process params here
    print res
    cur.execute('INSERT INTO grade (sendingID, error, result) VALUES (%s, %s, %s)', (sending[3], res['grade']['error'], res['grade']['result']))
    conn.commit()
    set_status('done')
    
def view_results():
    cur.execute('''
        SELECT
            code.code,
            code.params_info,
            sending.params,
            grade.result,
            grade.error
        FROM
            sending JOIN code ON sending.codeID = code.ID 
            JOIN grade ON grade.sendingID = sending.ID
    ''')
    for row in cur.fetchall():
        print row

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-1-467c122633b2>, line 6)

In [66]:
add_code('-rank(returns^3)', comment = 'test insert')

In [67]:
get_codes()

('-returns^3', None, 'test insert', 'python', 2, datetime.datetime(2017, 4, 15, 3, 15, 38, 768621, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)))
('-rank(returns^3)', None, 'test insert', 'python', 3, datetime.datetime(2017, 4, 15, 4, 41, 54, 783445, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=0, name=None)))


In [68]:
add_sending(3)

In [69]:
sending2 = get_sending_to_exec()
sending2

('-rank(returns^3)', None, None, 2)

In [51]:
get_websim(1)

('gosha-bor@yandex.ru', 'Cvwbn31sc', 'Georgy', None)


In [60]:
process_sending(sending, 1)

('gosha-bor@yandex.ru', 'Cvwbn31sc', 'Georgy', None)
<Response [200]>
{"status": true, "result": {"redirect": "/dashboard", "expiry": 14400, "timeout": 1800, "GAUID": "62c3eb61a6be7531aec6a64aed6522cd2cae1047adaf3f1a86f462cefe7e101a"}, "error": null}
<<class 'requests.cookies.RequestsCookieJar'>[<Cookie UID=62c3eb61a6be7531aec6a64aed6522cd2cae1047adaf3f1a86f462cefe7e101a for .worldquantchallenge.com/>, <Cookie WSSID="2|1:0|10:1492217407|5:WSSID|60:cVVrOFp4SWtHV3M5bE1YdklpcXFnOE5keC9OeE10T0UyazFWSkVrRW1SYz0=|ce41be4d56071fc3c5392c8ce7eceebc9ce55da06ad30e02b9f4db65d152d921" for .worldquantchallenge.com/>]>
{'UID': '62c3eb61a6be7531aec6a64aed6522cd2cae1047adaf3f1a86f462cefe7e101a', 'WSSID': '"2|1:0|10:1492217407|5:WSSID|60:cVVrOFp4SWtHV3M5bE1YdklpcXFnOE5keC9OeE10T0UyazFWSkVrRW1SYz0=|ce41be4d56071fc3c5392c8ce7eceebc9ce55da06ad30e02b9f4db65d152d921"'}
begin
<Response [200]>
{"status": true, "result": 9423826, "error": null}
9423826
sending all
<Response [200]>
{"status": true, "result": {"d

In [58]:
conn.rollback()

In [70]:
conn.commit()

In [62]:
view_results()

('-returns^3', None, None, 'Inferior', None)


In [74]:
cur.execute('INSERT INTO code (code, comment, author) VALUES (%s, %s, %s) RETURNING ID', ('abs(returns)', 'test return id', 'Georgy'))

In [75]:
cur.fetchall()

[(4,)]